In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas

data = pandas.read_csv("/content/drive/Othercomputers/내 노트북/File/spamVenv/files/spam.csv", encoding="latin1")
data = data[['v1', 'v2']].rename(columns={"v1":"isSpam", "v2":"content"}).drop_duplicates() #제목, 스팸여부 & 중복제거(결측치)
data["isSpam"] = data['isSpam'].replace(["ham", "spam"], [0, 1]) #범주형 -> 수치형
data["isSpam"].value_counts() #스팸 비율
print(f'스팸 : 정상 비율 : {data["isSpam"].value_counts()[0]/data["isSpam"].size} : {data["isSpam"].value_counts()[1]/data["isSpam"].size}')

스팸 : 정상 비율 : 0.873669955503966 : 0.12633004449603405


In [ ]:
from sklearn.model_selection import train_test_split

dataRatio  = 0.9
Xdata, _, Ydata, _ = train_test_split(data["content"], data["isSpam"], test_size=1 - dataRatio, random_state=123, stratify=data["isSpam"])

Xtrain, Xtest, Ytrain, Ytest = train_test_split(Xdata, Ydata, test_size=0.2, random_state=123, stratify=Ydata) #훈련데이터, 테스트데이터 나누기 (비율 8:2, 시드값 123?)
print(f'훈련데이터 정상 메일 = {round(Ytrain.value_counts()[0]/len(Ytrain) * 100)}%')
print(f'훈련데이터 스팸 메일 = {round(Ytrain.value_counts()[1]/len(Ytrain) * 100)}%')
print(f'총 훈련데이터 수 : {Ydata.size}개 중 {Ytrain.size}개, {round(Ytrain.size/Ydata.size*100)}%')
print(f'테스트데이터 정상 메일 = {round(Ytest.value_counts()[0]/len(Ytest) * 100)}%')
print(f'테스트데이터 스팸 메일 = {round(Ytest.value_counts()[1]/len(Ytest) * 100)}%')
print(f'총 테스트데이터 수 : {Ydata.size}개 중 {Ytest.size}개, {round(Ytest.size/Ydata.size*100)}%')
print(f'뽑은 데이터 비율 = {round(Ydata.size/data["isSpam"].size * 100)}%, 총 {data["isSpam"].size}개 중 {Ydata.size}개 선별.')




훈련데이터 정상 메일 = 87%
훈련데이터 스팸 메일 = 13%
총 훈련데이터 수 : 4652개 중 3721개, 80%
테스트데이터 정상 메일 = 87%
테스트데이터 스팸 메일 = 13%
총 테스트데이터 수 : 4652개 중 931개, 20%
뽑은 데이터 비율 = 90%, 총 5169개 중 4652개 선별.


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numpy import mean

tokenizer = Tokenizer() #토큰화 진행
tokenizer.fit_on_texts(Xdata) #각 단어를 숫자와 매핑시키는 집합 생성
encodedXtrain = tokenizer.texts_to_sequences(Xtrain) #각 문장의 단어들을 위 집합을 기반으로 [abc, cba] -> [3,39] 와 같이 변경 : encoding된 느낌

meanLen = round(mean([len(i) for i in Xdata])) #문장의 길이를 통일하기 위해 문장의 길이 평균값 -> 값 : 79정도

finalXtrain = pad_sequences(encodedXtrain, maxlen=meanLen) #입력 데이터 최종본 : 위에서 숫자로 이루어진 문장을 모두 같은 길이의 문장으로 만들어줌(0으로 채워넣는 pad sequence)
Xtrain.iloc[0], encodedXtrain[0], finalXtrain[0], print(f'문장의 길이 평균값 : {meanLen}')


문장의 길이 평균값 : 79


('Aiyar dun disturb u liao... Thk u have lots 2 do aft ur cupboard come...',
 [1752, 232, 1392, 6, 368, 235, 6, 16, 728, 20, 29, 573, 39, 7033, 59],
 array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0, 1752,  232,
        1392,    6,  368,  235,    6,   16,  728,   20,   29,  573,   39,
        7033,   59], dtype=int32),
 None)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten, SimpleRNN

model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, 32, input_length=meanLen)) #Embedding(단어 집합 길이 + 1, 출력 차원, 입력 최대 길이) -> 입력층
# model.add(SimpleRNN(32))
model.add(Flatten()) #다차원을 1차원으로 바꾸어 Dense에 들어갈 수 있도록 변환
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation="sigmoid")) #sigmoid함수를 활성화 함수로 하는 Dense, 출력 1개 -> 출력층
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
import time
startTime = time.time()
model.fit(finalXtrain, Ytrain, epochs=2, batch_size=32) # 전체 데이터를 2번, 32개씩 풀고 채점
endTime = time.time()
print(f'걸린 시간 : {endTime - startTime}초')

Epoch 1/2
117/117 [==============================] - 1s 4ms/step - loss: 0.2848 - accuracy: 0.8737
Epoch 2/2
117/117 [==============================] - 0s 4ms/step - loss: 0.1389 - accuracy: 0.9643
걸린 시간 : 1.6130201816558838초


In [ ]:
#모델을 평가하기 위해 test데이터를 전처리 하는 과정(encoding, pad ...)
encodedXtest = tokenizer.texts_to_sequences(Xtest)
finalXtest = pad_sequences(encodedXtest, maxlen=meanLen)

loss, accuracy = model.evaluate(finalXtest, Ytest, batch_size = 32)
print(f'정확도 : {accuracy*100}%')

30/30 [==============================] - 0s 2ms/step - loss: 0.1212 - accuracy: 0.9774
정확도 : 97.74436354637146%
